In [ ]:
!pip install phonetics
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install eng-to-ipa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for phonetics: filename=phonetics-1.0.5-py2.py3-none-any.whl size=8711 sha256=b717734b057c06b1ca0fa8b848e84e68b2a3ed8da44bc9d0a8ef793e838ba430
  Stored in directory: /root/.cache/pip/wheels/c2/c9/f4/5f43d3212d0aece0feced2484127ddb227ae43d57102aeb259
Successfully built phonetics
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 8.4 MB/s 
     |████████████████████████████████| 2.2 MB 52.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 9.8 MB/s 
  Created wheel for eng-to-ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822641 sha256=fa90cd0fc7128e5ffa6c64de9af8965db521be02c4

In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.8 MB/s 
     |████████████████████████████████| 7.6 MB 33.2 MB/s 
     |████████████████████████████████| 163 kB 45.9 MB/s 
     |████████████████████████████████| 1.3 MB 45.9 MB/s 


In [ ]:
import xml.etree.ElementTree as ET
import random
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
path_test = "/content/subtask2-heterographic-test.xml"
path_gold = "/content/subtask2-heterographic-test.gold"

In [ ]:
pun_instances = {}
classes = {}
locations = {}
max_sent_len = None

tree = ET.parse(path_test)
root = tree.getroot()

for child in root:
    line = []
    idx = child.attrib["id"]
    for kid in child:
        line.append(kid.text)
    pun_instances[idx] = line

with open(path_gold) as gold:
    lines = gold.readlines()
    for line in lines:
        token = line.strip().split("\t")
        sub_tokens = token[1].split("_")
        locations[token[0]] = sub_tokens[2]

all_data = []

for idx in pun_instances.keys():
    sentence = " ".join(pun_instances[idx])
    pun_word = pun_instances[idx][int(locations[idx]) - 1]
    pun_location = int(locations[idx]) - 1    
    all_data.append({"sentence": sentence, "location": pun_location})

In [ ]:
pp.pprint(random.sample(all_data, 10))

[ { 'location': 11,
    'sentence': "If you want those ashes , you ' ll have to urn them ."},
  {'location': 6, 'sentence': "The shy stripper couldn ' t bare it on stage ."},
  { 'location': 14,
    'sentence': "She was only a Quarryman ' s daughter , but she took "
                'everything for granite .'},
  {'location': 5, 'sentence': 'A church fair is a bazaar experience .'},
  { 'location': 16,
    'sentence': 'I was given a shampoo and trim by a Pakistani in Liverpool , '
                'said Tom in Urdu .'},
  { 'location': 12,
    'sentence': 'If an actress has a screaming role , can we say she eeks out '
                'a living ?'},
  {'location': 5, 'sentence': 'An unemployed logger is a would worker .'},
  { 'location': 14,
    'sentence': 'Communism is a complicated thing . You must think about from '
                'all sorts of Engels .'},
  { 'location': 8,
    'sentence': 'Another batch of shells for me ! Tom clamored .'},
  {'location': 3, 'sentence': "My name is

# Identifying Source and Target

In [ ]:
import nltk
import string
import re
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from fuzzywuzzy import fuzz
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('cmudict')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


True

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
def remove_punctuation(text):
    pf = "".join([i for i in text if i not in string.punctuation])
    return pf

def turn_to_lowercase(text):
  return text.lower()

wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
  lemm_text = "".join([wordnet_lemmatizer.lemmatize(word) for word in text])
  return lemm_text

def tokenization(text):
    return text.split(" ")

def remove_stopwords(wordList):
    output = [w for w in wordList if w not in stopwords]
    return output

In [ ]:
sentence = "If you want those ashes , you have to urn them ."

In [ ]:
s_transformed = remove_stopwords(tokenization(lemmatizer(turn_to_lowercase(remove_punctuation(sentence)))))
s_transformed = [ x for x in s_transformed if x]
s_transformed

['want', 'ashes', 'urn']

In [ ]:
arpabet = nltk.corpus.cmudict.dict()

In [ ]:
s_arp = []
for w in s_transformed:
  try:
    s_arp.append(arpabet[w][0])
  except:
    s_transformed.remove(w)
print(s_arp)
print(s_transformed)

[['W', 'AA1', 'N', 'T'], ['AE1', 'SH', 'AH0', 'Z'], ['ER1', 'N']]
['want', 'ashes', 'urn']


In [ ]:
threshold = 90

In [ ]:
results = []
for word in s_arp:
  itemList = []
  for w, a in arpabet.items():
    if fuzz.ratio(word, a) > threshold and w not in s_transformed:
      itemList.append(w)
  results.append(itemList)
results

[['bondt',
  'bonte',
  'font',
  'fonte',
  'mont',
  'pont',
  'ponte',
  'waft',
  'wand',
  'wonk'],
 ['ashen', 'asses'],
 ['earn', 'erne']]

# WordNet Senses

In [ ]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.tag import pos_tag
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
sent = "If you want those ashes , you have to urn them ."

In [ ]:
source = "urn"
target = "earn"

In [ ]:
# Create function to map
s_mapping = wn.NOUN
t_mapping = wn.VERB

In [ ]:
res_pos = None
sent_pos = pos_tag(sent.split())
for w, pos in sent_pos:
  if w == source:
    res_pos = pos[0].lower()
res_pos

'v'

In [ ]:
src_set = wn.synsets(source, pos=s_mapping)
tgt_set = wn.synsets(target, pos=t_mapping)
print(src_set[0].definition())
print(tgt_set[0].definition())
print(src_set[0].path_similarity(tgt_set[0]))

a large vase that usually has a pedestal or feet
earn on some commercial or business transaction; earn as salary or wages
0.07142857142857142


In [ ]:
wn.synsets('knows')

[Synset('know.n.01'),
 Synset('know.v.01'),
 Synset('know.v.02'),
 Synset('know.v.03'),
 Synset('know.v.04'),
 Synset('know.v.05'),
 Synset('acknowledge.v.06'),
 Synset('know.v.07'),
 Synset('sleep_together.v.01'),
 Synset('know.v.09'),
 Synset('know.v.10'),
 Synset('know.v.11')]

# Sound Similarity

In [ ]:
!pip install phonetics
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install eng-to-ipa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 4.4 MB/s 
  Created wheel for eng-to-ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822641 sha256=22d7e2bc6f257ba57f7a85395222ac149052db9219d9cfcd6a259e23731b2642
  Stored in directory: /root/.cache/pip/wheels/96/c0/dd/aeddfbebc2c3301c3dd09670d9954b0574ac4cd982664c1110
Successfully built eng-to-ipa


In [ ]:
from fuzzywuzzy import fuzz
import phonetics
import eng_to_ipa as ipa
import nltk

### Double Metaphone

In [ ]:
word_1_dmeta = phonetics.dmetaphone('propane')
print(word_1_dmeta)
word_2_dmeta = phonetics.dmetaphone('profane')
print(word_2_dmeta)
print(fuzz.ratio(word_1_dmeta, word_2_dmeta))

('PRPN', '')
('PRFN', '')
92


### IPA

In [ ]:
word_1_ipa = ipa.convert('propane')
print(word_1_ipa)
word_2_ipa = ipa.convert('profane')
print(word_2_ipa)
print(fuzz.ratio(word_1_ipa, word_2_ipa))

ˈproʊˌpeɪn
proʊˈfeɪn
74


### ARPABET

In [ ]:
arpabet = nltk.corpus.cmudict.dict()
word_1_arp = arpabet['propane']
print(word_1_ipa)
word_2_arp = arpabet['profane']
print(word_2_ipa)
print(fuzz.ratio(word_1_arp, word_2_arp))

ˈproʊˌpeɪn
proʊˈfeɪn
92
